In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
def binder_url(org, repo):
    return f'https://notebooks.gesis.org/binder/v2/gh/{org}/{repo}/master'

In [3]:
def ts_to_dt(ts):
    return datetime.utcfromtimestamp(ts)

# The most popular repositories

In [4]:
def query(time_range):
    query = 'binderhub_launch_time_seconds_count{}[{}]'
    query_selectors = "{status='success'}"
    query = query.format(query_selectors, time_range)
    print(query)
    resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
    data = resp.json()['data']['result']
    return data

In [5]:
def makedf(data, start_time):
    df = {'name': [], 'org': [], 'provider': [], 'launches': [], 'repo_url': [], 'binder_url': []}
    for container in data:
        repo_url = container['metric']['repo']
        pref, org, repo = container['metric']['repo'].rsplit('/', 2)
        provider = pref.replace('https://', '')
        
        values = [int(ii[1]) for ii in container['values']]
        first_value_ts = container['values'][0][0]
        first_value_dt = datetime.utcfromtimestamp(first_value_ts)
        # first_value = values[0]
        #if first_value_dt > start_time:
        #    assert first_value == 1, f'{org}/{repo}---{first_value}---{first_value_dt}---{start_time}'
        if first_value_dt > start_time:
            # NOTE first_value can be > 1 if there are simultaneous launches
            print(repo, first_value_dt, start_time)
            launches = max(values)
        else:
            launches = max(values) - min(values)
                
        # print(repo_url, launches, container['metric']['status'], container['metric']['retries'])
        if repo in df['name']:
            # same repo can have status success with different retries value
            i = df['name'].index(repo)
            df['launches'][i] += launches
        else:            
            df['name'].append(repo)
            df['org'].append(org)
            df['provider'].append(provider)
            df['launches'].append(launches)
            df['repo_url'].append(repo_url)
            df['binder_url'].append(binder_url(org, repo))
    df = pd.DataFrame(df)
    df = df.drop_duplicates(['name'])
    df = df.groupby(['name', 'org', 'provider','repo_url', 'binder_url']).sum().reset_index().sort_values('launches', ascending=False)
    df['log_launches'] = df['launches'].apply(np.log)
    return df

## The most popular repositories in last hour

In [6]:
hour = 24
data = query(f'{hour}h')
start_time = datetime.utcnow() - timedelta(hours=hour)
df = makedf(data, start_time)

binderhub_launch_time_seconds_count{status='success'}[24h]
BIGSSS 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374
gesis-meta-analysis-2018 2018-10-11 08:46:44.244000 2018-10-11 06:43:06.161374
bokeh 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374
r 2018-10-11 22:26:44.244000 2018-10-11 06:43:06.161374
PythonDataScienceHandbook 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374
binder-stats 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374
flow 2018-10-11 10:59:44.244000 2018-10-11 06:43:06.161374
workshop_girls_day 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374
ligo-binder 2018-10-11 06:43:44.244000 2018-10-11 06:43:06.161374


In [7]:
df.style.format({'repo_url':lambda x: f'<a target="_blank" href="{x}">repo url</a>', 
                 'binder_url': lambda x: f'<a target="_blank" href="{x}">binder url</a>'})

,name,org,provider,repo_url,binder_url,launches,log_launches
1,PythonDataScienceHandbook,gesiscss,github.com,repo url,binder url,20,2.99573
8,workshop_girls_day,gesiscss,github.com,repo url,binder url,8,2.07944
2,binder-stats,gesiscss,github.com,repo url,binder url,6,1.79176
6,ligo-binder,minrk,github.com,repo url,binder url,5,1.60944
0,BIGSSS,JuKo007,github.com,repo url,binder url,1,0
3,bokeh,binder-examples,github.com,repo url,binder url,1,0
4,flow,gesiscss,github.com,repo url,binder url,1,0
5,gesis-meta-analysis-2018,berndweiss,github.com,repo url,binder url,1,0
7,r,binder-examples,github.com,repo url,binder url,1,0
